#Draft

In [ ]:
'''
			to_modify = modified_plots
			adverb = outer_level[0][1][0]
			noun1,noun2 = outer_level[0][2][0],outer_level[0][2][1]
			output1 = interactive.get_conceptnet_sequence(noun1, self.comet_model, self.sampler, self.data_loader, self.text_encoder, relation)[0]
			output2 = interactive.get_conceptnet_sequence(noun2, self.comet_model, self.sampler, self.data_loader, self.text_encoder, relation)[0]
			manipulation1 = " ".join(noun1,adverb,output2) 
			manipulation2 = " ".join(noun2,adverb,output1) 

			for i1 in modified_plots:
				for i2 in i1:
					for i3 in i2:
			to_modify_flat = []
			for t_m in to_modify: to_modify_flat.extend(t_m)
			
			noun1_idx = to_modify.index(noun1)
			noun2_idx = to_modify.index(noun2)
			
			to_modify[noun1_idx] = " ".join(noun1,adverb,output2)
			to_modify[noun2_idx] = " ".join(noun2,adverb,output1)

			modified_plots[candidate_loc] = "\t".join(to_modify)
			candidates = outer_level'''

In [ ]:
#sys.path.append('/content/drive/My Drive/Plot-guided-Coherence-Evaluation-main/src')
#!python src/main.py --experiment_type conceptnet --experiment_num 0

#from google.colab import files
#src = list(files.upload().values())[0]
#open('storyline_manipulation_ROC.py','wb').write(src)
#import storyline_manipulation_ROC
'''src = list(files.upload().values())[0]
open('src/data/data.py','wb').write(src)
import data as data

src = list(files.upload().values())[0]
open('src/data/config.py','wb').write(src)
import config as cfg

src = list(files.upload().values())[0]
open('storyline_manipulation_ROC.py','wb').write(src)
import functions as interactive'''

#Pip packages

In [1]:
!pip install tensorboardX
!pip install pyinflect
!pip install ftfy==5.1
!pip install fairseq

     |████████████████████████████████| 124 kB 5.1 MB/s 
     |████████████████████████████████| 703 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 460 kB/s 
  Created wheel for ftfy: filename=ftfy-5.1-py3-none-any.whl size=39219 sha256=aa7e68d57acb5570de8770400d01da0585205fbf04bb18be94ab3dbd8fe5a3bc
  Stored in directory: /root/.cache/pip/wheels/80/0a/4e/64bbd722ea7bed148b8ef1516dbae348d50b296822e7c1ef41
Successfully built ftfy
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 90 kB 11.4 MB/s 
     |████████████████████████████████| 145 kB 64.6 MB/s 
     |████████████████████████████████| 112 kB 60.3 MB/s 
     |████████████████████████████████| 74 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 59.3 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=a7347c8b07a28a4699937ebede4568461493b9cac32d33011ff0d80d9bf47af3
  Stored in directory: /root/.

#Imports

In [7]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/NLP_Project/')

from base_project.storyline_manipulation_ROC import Plt_manipulations as Plt_manipulations_ROC
from base_project.storyline_manipulation_WP import Plt_manipulations as Plt_manipulations_WP

from new_storyline_manipulation_WP import Plt_manipulations as new_Plt_manipulations_WP
#from new_storyline_manipulation_ROC import Plt_manipulations as new_Plt_manipulations_ROC

#from base_project.make_AF_input_ROC import AF_Data_Generation as AF_Data_Generation_ROC
#from base_project.make_AF_input_WP import AF_Data_Generation as AF_Data_Generation_WP

import csv
import json
import torch
import os
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import math
import spacy
import pyinflect
import argparse 
import src.data.data as data
import src.data.config as cfg
import src.interactive.functions as interactive
from fairseq.models.bart import BARTModel
nlp = spacy.load('en_core_web_sm')
np.random.seed(100)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


#Helper functions

In [8]:
def separete_prompts():
  data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/WP/WP_Eval/ManPlts"
  data_dir2 = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_ManPlts"  
  prompt_file = open(os.path.join(data_dir, "WP_valid_manipulpltsprompt"), 'r')
  prompts = open(os.path.join(data_dir2, "prompts_valid"), 'w')
  for line in prompt_file.readlines():
    prompt = line.split('<EOL>')[0].strip()
    prompts.write(prompt + '\n')
  prompts.close();prompt_file.close()

def join_prompt_manPlts():
  data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_ManPlts"  
  prompts = open(os.path.join(data_dir, "prompts_valid"), 'r')
  man_plts = open(os.path.join(data_dir, "WP_valid_manipulated_plts"), 'r')
  prompts_plots = open(os.path.join(data_dir, "WP_valid_manipulpltsprompt"), 'w')
  prompts_lines,man_plts_lines = prompts.readlines(),man_plts.readlines()
  print(len(prompts_lines),len(man_plts_lines))
  assert len(prompts_lines)==len(man_plts_lines)
  for pr,plt in zip(prompts_lines,man_plts_lines):
    prompt = pr.split("\n")[0].strip()
    plot = plt.split("\n")[0].strip()
    prompts_plots.write(" <EOL> ".join([prompt,plot]) + '\n')
  prompts.close();man_plts.close()
  prompts_plots.close()

In [9]:
#separete_prompts()
join_prompt_manPlts()

2399 2399


#ROC

##Step 1

In [ ]:
#print(torch.__version__)
#parser = argparse.ArgumentParser()
#parser.add_argument("--COMET_model_file", type=str, default="pretrained_models/conceptnet_pretrained_model.pickle")
#parser.add_argument("--device", type=str, default="cpu")
#parser.add_argument("--COMET_sampling_algorithm", type=str, default="beam-10")
#parser.add_argument("--data_dir", type=str, default="Data_/ROC/ROC_Eval")
#parser.add_argument("--fname", type=str, default="Rocstories_train")

#args = parser.parse_args()
COMET_model_file = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/pretrained_models/conceptnet_pretrained_model.pickle"
device = 0 #torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
COMET_sampling_algorithm = "beam-5"
data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/Data_/ROC/ROC_Eval"
fname = "Rocstories_train"

fr_stories = open(os.path.join(data_dir, fname), 'r')
lines = fr_stories.readlines()
stories = []
set_plots =[]

for line in lines:
	storyplots =  line.split('<EOT>')[1].split('<EOL>')[0].strip()
	story_content =  line.split('<EOL>')[1].split('@')[0].strip()
	stories.append(story_content)
	set_plots.append(storyplots)
		
plt_changes = Plt_manipulations1(COMET_model_file, COMET_sampling_algorithm, device)


num_gens= 1
if not os.path.isdir(data_dir+'/ManPlts/'):
	os.mkdir(data_dir+'/ManPlts/')
fw_plts = open(os.path.join(data_dir+'/ManPlts', fname +'_manipulated_plts_NEW'), 'w')
	
	
for ind, story_plots in enumerate(set_plots):
	print('******************{}****************'.format(ind))
	print('STORY PLOTS:\t'+story_plots)
	print('STORY:\t'+stories[ind])
	for i in range(num_gens):
		manipulated_story_plts = story_plots
		num_changes = np.random.choice([2,3], size=1, replace=False)[0]	
		ind_technique_apply = np.random.choice([4,5,6], size=num_changes, replace=False)
		print('number of changes {}'.format(num_changes))
		print('the techniques to apply is {}'.format(ind_technique_apply))
		for tech_ind in ind_technique_apply:
			if tech_ind ==0:
				manipulated_story_plts = plt_changes.insert_antonym(manipulated_story_plts)
				print('after antonym insertion:\n{}'.format(manipulated_story_plts))
			elif tech_ind ==1:
				manipulated_story_plts = plt_changes.repetition(manipulated_story_plts)
				print('after repetitions:\n{}'.format(manipulated_story_plts))
			elif tech_ind ==2:
				manipulated_story_plts = plt_changes.notlogic_ordered(manipulated_story_plts, stories[ind])
				print('after noot logically order:\n{}'.format(manipulated_story_plts))
			elif tech_ind ==3:
				manipulated_story_plts = plt_changes.plt_random_insertion(manipulated_story_plts, set_plots)
				print('after random isertion:\n{}'.format(manipulated_story_plts))
			elif tech_ind ==4:
				manipulated_story_plts = plt_changes.insert_antonym_2(manipulated_story_plts)
				print('after antonym insertion 2:\n{}'.format(manipulated_story_plts))
			elif tech_ind ==5:
				manipulated_story_plts = plt_changes.contradiction_LogicalReordereing(manipulated_story_plts, stories[ind])
				print('after contradiction_LogicalReordereing:\n{}'.format(manipulated_story_plts))
			elif tech_ind == 6:
				manipulated_story_plts = plt_changes.random_deletion(manipulated_story_plts)
				print('after random_deletion:\n{}'.format(manipulated_story_plts))	
		fw_plts.write(manipulated_story_plts.strip() + '\n')
	print('_________________________________________')
fw_plts.close()

OPT:  DD{'net': DD{'model': 'transformer', 'nL': 12, 'nH': 12, 'hSize': 768, 'edpt': 0.1, 'adpt': 0.1, 'rdpt': 0.1, 'odpt': 0.1, 'pt': 'gpt', 'afn': 'gelu', 'init': 'pt', 'vSize': 40545}, 'mle': 0, 'dataset': 'conceptnet', 'train': DD{'static': DD{'exp': 'generation', 'seed': 123, 'l2': 0.01, 'vl2': True, 'lrsched': 'warmup_linear', 'lrwarm': 0.002, 'clip': 1, 'loss': 'nll', 'b2': 0.999, 'b1': 0.9, 'e': 1e-08}, 'dynamic': DD{'lr': 1e-05, 'bs': 64, 'optim': 'adam', 'epoch': 15500}}, 'model': 'transformer', 'exp': 'generation', 'data': DD{'rel': 'language', 'trainsize': 100, 'devversion': '12', 'maxe1': 10, 'maxe2': 15}, 'eval': DD{'bs': 1, 'smax': 40, 'sample': 'greedy', 'numseq': 1, 'gs': 'full', 'es': 'full', 'categories': None}, 'trainer': 'iteration', 'cycle': 500, 'iters': 100000}
/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/data/conceptnet/processed/generation/rel_language-trainsize_100-devversion_12-maxe1_10-maxe2_15-maxr_5.pickle
Load

/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/src/evaluate/sampler.py:275: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  for i in beam_losses]
/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/src/evaluate/sampler.py:276: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_m

Streaming output truncated to the last 5000 lines.
the techniques to apply is [5 4]
after contradiction_LogicalReordereing:
watching sports	#	players	field	#	player threw	#	team tackled	player	#	team lined
after antonym insertion 2:
watching sports	#players	rink#	players	field	#players	rink#	player threw	#	team tackled	player	#	team lined
_________________________________________
******************23325****************
STORY PLOTS:	loved animals	#	greatest dream	real safari	#	parents surprised	safari trip	birthday	#	animal	loved	safari	#	sunburn
STORY:	</s> will was a boy who really loved animals . </s> will 's greatest dream was to go on a real safari . </s> will 's parents surprised him with a safari trip for his birthday . </s> will saw every animal he loved while on the safari . </s> will also got a pretty good sunburn during the trip , too .
number of changes 3
the techniques to apply is [4 6 5]
after antonym insertion 2:
loved animals	#	greatest dream	real safari	#parents surpris

##Step 2

In [ ]:
'''parser = argparse.ArgumentParser()
parser.add_argument("--bart_model_path", type=str, default="Models/Ft_BART_Story_Generator/WP/", help="model path including finetuned BART model as the conditional LM")
parser.add_argument("--data_path", type=str, default='Data/WP/WP_Eval/', help='data path for WP_Eval data')
parser.add_argument("--json_file", type=str, default='WP_AF_input.json', help='json input files')
parser.add_argument("--num_negative_samples", type=int, default=6, help="number of negative (implausible) samples to generate for each plausible story")
parser.add_argument("--batch_size", type=int, default=20, help='batch size to generate samples')
parser.add_argument("--max_len_b", type=int, default=750, help='max length of stories')
parser.add_argument("--min_len", type=int, default=10, help='min length of stories')
parser.add_argument("--sampling_topk", type=int, default=50, help='topk sampling')
parser.add_argument("--temperature", type=float, default=0.8, help='temperature value')
parser.add_argument("--beam", type=float, default=4, help='beam size')'''

bart_model_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/Models/Ft_BART_Story_Generator/WP/"
data_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/Data_/WP/WP_Eval/"
json_file = "WP_AF_input.json"
num_negative_samples = 6
batch_size = 20
max_len_b = 750
min_len = 10
sampling_topk = 50
temperature = 0.8
beam = 4
args = {"bart_model_path":bart_model_path,"data_path":data_path,"json_file":json_file,
        "num_negative_samples":num_negative_samples,"batch_size":batch_size,
        "max_len_b":max_len_b,"min_len":min_len,"sampling_topk":sampling_topk,
        "temperature":temperature,"beam":beam}

af = AF_Data_Generation_WP(bart_model_path)
for  file_type in ['train', 'valid']:
  gt_stories=data_path+'WP_{}.target'.format(file_type) #file including ground truth stories
  man_plts=data_path+'ManPlts/WP_{}_manipulated_plts'.format(file_type) #file including manipulated plots
  pltprompt=data_path+'ManPlts/WP_{}_manipulpltsprompt'.format(file_type) #this file includes manipulated plots with the orginal prompts
  lines_plots_prompts = open(pltprompt, 'r').readlines()
  plots_prmpts={} #is a dictionary including the prompts for each manipulated plts
  for line in lines_plots_prompts:
  	plts = line.split('\n')[0].split('<EOL>')[1].strip()
  	if plts not in plots_prmpts:
  		plots_prmpts[plts] = line.split('<EOL>')[0].strip()
  fr_gt = open(gt_stories, 'r')
  lines_gt_stories = fr_gt.readlines()
  fr = open(man_plts, 'r')
  manipulated_plots = fr.readlines()
  manipulated_plots_new = []
  prompts=[]      
  lines_gt_new = []
  for ind, plt in enumerate(manipulated_plots[:5]):
  	plt  = plt.split('\n')[0]
  	if plt in plots_prmpts.keys():
  		manipulated_plots_new.append(plt)
  		lines_gt_new.append(lines_gt_stories[ind])
  		#add two of manipulated plts' prompt (used in create_json_AF_input function)
  		prompts.append(plots_prmpts[plt])
  		prompts.append(plots_prmpts[plt])
  manipulated_plots = manipulated_plots_new
  lines_gt_stories = lines_gt_new
  gt_stories=[]
  num_sents_gt_stories=[]
  for ind_line, story in  enumerate(lines_gt_stories):
  	gt_story = lines_gt_stories[ind_line].split('\n')[0]
  	gt_stories.append(gt_story)
  	num_sents_gt_stories.append(len(gt_story.split('</s>')))
  #Generate args.num_negative_samples different tsv files each including gt_stories as positive and generated negative stories as implausible ones
  for ind in range(num_negative_samples):
    #print(args['data_path'])
    af.generate_implausible_stories(args, gt_stories, manipulated_plots, num_sents_gt_stories, ind,file_type)
af.create_json_AF_input(args, prompts)

##Step 4-5

In [ ]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--data_path", type=str, default='Data/WP/WP_Eval/', help='data path for WP_Eval data')
#parser.add_argument("--output_AF", type=str, default='AF_output.json', help='The result file from AF')
#parser.add_argument("--train_output", type=str, default='AF_ManPlts_train.tsv', help='The training file resulted from AF applied on WP manipulated plots to be used for training the evaluator')
#parser.add_argument("--valid_output", type=str, default='AF_ManPlts_valid.tsv', help='The validation file resulted from AF applied on WP manipulated plots to be used for validating the evaluator')
#parser.add_argument("--test_output", type=str, default='AF_ManPlts_test.tsv', help='The testing file resulted from AF applied on WP manipulated plots to be used for testing the evaluator')
#args = parser.parse_args()
data_path = '/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/Data_/WP/WP_Eval/'
output_AF = 'AF_output.json'
train_output = 'AF_ManPlts_train.tsv'
valid_output = 'AF_ManPlts_valid.tsv'
test_output = 'AF_ManPlts_test.tsv'

fr = open(os.path.join(data_path, output_AF), 'r')
fw_train = open(os.path.join(data_path, 'adv_ManPlts/'+train_output), 'w')
fw_valid = open(os.path.join(data_path, 'adv_ManPlts'+valid_output), 'w')
fw_test = open(os.path.join(data_path, 'adv_ManPlts'+test_output), 'w')
tsv_train = csv.writer(fw_train, delimiter='\t', lineterminator='\n')
tsv_valid = csv.writer(fw_valid, delimiter='\t', lineterminator='\n')
tsv_test = csv.writer(fw_test, delimiter='\t', lineterminator='\n')

list_ctx = []
list_gt = []
list_gens = []
num_stories= 0
for line in fr:
	line = json.loads(line)
	list_ctx.append(line['ctx'])
	list_gt.append(line['gt_detok'])
	gens = []
	#line['assignment'][-1] shows the index of most challenging generated stories based on the applied AF
	sel_inds = line['assignment'][-1]
	for ind in sel_inds:
		gens.append(line['gens'][ind])
	list_gens.append(gens)
 
num_stories=len(list_ctx)
num_train_stories = math.ceil((60*num_stories)/100)
num_valid_stories = math.ceil((20*num_stories)/100)

start_ind={'train':0, 'valid':num_train_stories, 'test': num_train_stories + num_valid_stories}
for mode in ['train', 'valid', 'test']:
	st_ind = start_ind[globals()['{}'.format(mode)]]
	globals()['{}_ctx'.format(mode)] = list_ctx[globals()[st_ind:st_ind+'num_{}_stories'.format(mode)]]
	globals()['{}_gt'.format(mode)] = list_gt[globals()[st_ind:st_ind+'num_{}_stories'.format(mode)]]
	globals()['{}_gens'.format(mode)] = list_gens[globals()[st_ind:st_ind+'num_{}_stories'.format(mode)]]

	
for mode in ['train', 'valid', 'test']:
	line_ind = 0
	for ind, gt in enumerate(globals()['{}_gt'.format(mode)]):
		globals()['tsv_{}'.format(mode)].writerow([line_ind, 1, line_ind , gt])
		line_ind+=1
		for gen in globals()['{}_gens'.format(mode)][ind]:
			globals()['tsv_{}'.format(mode)].writerow([line_ind, 0, line_ind, gen])
			line_ind+=1

FileNotFoundError: ignored

#WP

##Step1

In [5]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--COMET_model_file", type=str, default="/content/drive/My Drive/Colab Notebooks/NLP_Project/Plot-guided-Coherence-Evaluation-main/pretrained_models/conceptnet_pretrained_model.pickle")
#parser.add_argument("--device", type=str, default="cpu")
#parser.add_argument("--COMET_sampling_algorithm", type=str, default="beam-10")
#parser.add_argument("--data_dir", type=str, default="Data_/WP/WP_Eval")
#parser.add_argument("--fname", type=str, default="WP_train")
#args = parser.parse_args()
COMET_model_file = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/pretrained_models/conceptnet_pretrained_model.pickle"
device = 0 #torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
COMET_sampling_algorithm = "beam-1"
data_dir = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/WP/WP_Eval"
data_dir2 = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP"
fname = "WP_valid"#"WP_train"
	
lines_plots = open(os.path.join(data_dir, fname +'.source'), 'r').readlines()#source - plots
lines_stories = open(os.path.join(data_dir, fname+'.target'), 'r').readlines()
assert len(lines_plots)==len(lines_stories)

set_plots = [line.split('\n')[0].strip() for line in lines_plots]
stories = [lines_stories[ind].split('\n')[0].strip() for ind in range(len(lines_plots))]

output_file = fname +'_manipulated_plts'
man_plts_folder = data_dir2+'/WP_ManPlts'
if not os.path.isdir(man_plts_folder):
	os.mkdir(man_plts_folder)
fw_plts = open(os.path.join(man_plts_folder, output_file), 'w')


num_gens= 1	
#plt_changes = Plt_manipulations_WP(COMET_model_file, COMET_sampling_algorithm, device)
print(COMET_model_file)
new_plt_changes = new_Plt_manipulations_WP(COMET_model_file, COMET_sampling_algorithm, device)
for ind, story_plots in enumerate(set_plots):
  print('******************{}****************'.format(ind))
  print('STORY PLOTS:\t'+story_plots,"\n")
  print('STORY:\t'+stories[ind],"\n")
  for i in range(num_gens):
    manipulated_story_plts = story_plots
    num_changes = np.random.choice([2,3], size=1, replace=False)[0]	
    ind_technique_apply = np.random.choice([4,6,7], size=num_changes, replace=False)
    print('number of changes {}'.format(num_changes))
    print('the techniques to apply is {}\n'.format(ind_technique_apply))
    for tech_ind in ind_technique_apply:
      if tech_ind ==0:
        manipulated_story_plts = plt_changes.insert_antonym(manipulated_story_plts)
        print('after antonym insertion {}'.format(manipulated_story_plts))
      elif tech_ind ==1:
        manipulated_story_plts = plt_changes.repetition(manipulated_story_plts)
        print('after repetitions {}'.format(manipulated_story_plts))
      elif tech_ind ==2:
        manipulated_story_plts = plt_changes.notlogic_ordered(manipulated_story_plts, stories[ind])
        print('after not logically order {}'.format(manipulated_story_plts))
      elif tech_ind ==3:
        manipulated_story_plts = plt_changes.plt_random_insertion(manipulated_story_plts, set_plots)
        print('after random isertion {}'.format(manipulated_story_plts))
      elif tech_ind ==4:
        manipulated_story_plts = new_plt_changes.insert_antonym_2(manipulated_story_plts)
        print('after antonym insertion 2 \n{}'.format(manipulated_story_plts))
      elif tech_ind ==5:
        manipulated_story_plts = new_plt_changes.contradiction_LogicalReordereing(manipulated_story_plts, stories[ind])
        print('after contradiction_LogicalReordereing {}'.format(manipulated_story_plts))
      elif tech_ind ==6:
        manipulated_story_plts = new_plt_changes.random_deletion(manipulated_story_plts)
        print('after random_deletion:\n{}'.format(manipulated_story_plts))
      elif tech_ind ==7:
        manipulated_story_plts = new_plt_changes.change_location(manipulated_story_plts)
        print('after change locations:\n{}'.format(manipulated_story_plts))
		#print(manipulated_story_plts)
    fw_plts.write(manipulated_story_plts.strip() + '\n')
  print('_________________________________________')
fw_plts.close()

/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/pretrained_models/conceptnet_pretrained_model.pickle
/content/drive/My Drive/Colab Notebooks/NLP_Project/data/conceptnet/processed/generation/rel_language-trainsize_100-devversion_12-maxe1_10-maxe2_15-maxr_5.pickle
Loading data from: /content/drive/My Drive/Colab Notebooks/NLP_Project/data/conceptnet/processed/generation/rel_language-trainsize_100-devversion_12-maxe1_10-maxe2_15-maxr_5.pickle
31
******************0****************
STORY PLOTS:	answering	machine	</s>	</s>	you	know	our office	received	a crank call	from this number	playing	</s>	sight	corrected	dial	a specific number	</s>	want	to see anymore	you	ought	to look into that	</s>	you	call	we	recommend	some trauma specialists	</s>	Maria	sighed	</s>	fix	six years of disability 

STORY:	Maria hit the voicemail button on her answering machine . </s> `` Mrs. Tenenfield , this is Dr. Routabagh calling . </s> I wanted to let you know our office received a crank call from 

/content/drive/My Drive/Colab Notebooks/NLP_Project/src/evaluate/sampler.py:275: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  for i in beam_losses]
/content/drive/My Drive/Colab Notebooks/NLP_Project/src/evaluate/sampler.py:276: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ended = ended.index_select(0, top_beam_idxs // self.opt.eval

Streaming output truncated to the last 5000 lines.

after random_deletion:
blackened	degraded	into static and	</s>	dying	from ago	fading	</s>	think	it	</s>	</s>	the	comes	sizzles	a	rises	</s>	live	</s>	It	gets	sad	</s>	wish	go back	</s>	look	pictures	listen	songs	</s>	know	what	</s>	I	'	glad	</s>	beaten	battered	</s>	left	</s>	histories	gone	burnt	memories	scarred	</s>	</s>	been	important	so	made	</s>	speak	God	</s>	read	</s>	think	it	</s>	seem	speak	to completely different species	</s>	Men	chained	</s>	rights	waived	</s>	pay	being	product	</s>	what	old folk	mean	</s>	Is	cruel	</s>	look	survive	</s>	fade	</s>	look	pictures	see	pristine world	</s>	</s>	</s>	clean	living	your	</s>	The destruction	started	loose policy	I	told	</s>	</s>	</s>	set	on	</s>	Our	collapsing	</s>	it	happen	</s>	What	caused	all	</s>	they	follow	</s>	binding	document	</s>	that	stopped	man	from being so destructive	</s>	know	</s>	</s>	Life	shortening	</s>	rain	falls	that	grow	</s>	dark grey	keeps	building and	</s>	</

##Step 3

In [ ]:
'''parser = argparse.ArgumentParser()
parser.add_argument("--bart_model_path", type=str, default="Models/Ft_BART_Story_Generator/WP/", help="model path including finetuned BART model as the conditional LM")
parser.add_argument("--data_path", type=str, default='Data/WP/WP_Eval/', help='data path for WP_Eval data')
parser.add_argument("--json_file", type=str, default='WP_AF_input.json', help='json input files')
parser.add_argument("--num_negative_samples", type=int, default=6, help="number of negative (implausible) samples to generate for each plausible story")
parser.add_argument("--batch_size", type=int, default=20, help='batch size to generate samples')
parser.add_argument("--max_len_b", type=int, default=750, help='max length of stories')
parser.add_argument("--min_len", type=int, default=10, help='min length of stories')
parser.add_argument("--sampling_topk", type=int, default=50, help='topk sampling')
parser.add_argument("--temperature", type=float, default=0.8, help='temperature value')
parser.add_argument("--beam", type=float, default=4, help='beam size')'''

bart_model_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Models/Ft_BART_Story_Generator/WP/"
data_path = "/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_ManPlts/"
data_path2 = "/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/WP/WP_Eval/"
json_file = "WP_AF_input.json"
num_negative_samples = 6
batch_size = 20
max_len_b = 750
min_len = 10
sampling_topk = 50
temperature = 0.8
beam = 4
#args = {"bart_model_path":bart_model_path,"data_path":data_path,"json_file":json_file,
#        "num_negative_samples":num_negative_samples,"batch_size":batch_size,
#        "max_len_b":max_len_b,"min_len":min_len,"sampling_topk":sampling_topk,
#        "temperature":temperature,"beam":beam}

bart = BARTModel.from_pretrained(bart_model_path,
                                 checkpoint_file='checkpoint_best.pt',
                                 data_name_or_path=bart_model_path+'wp_kw_story-bin')
bart.cuda()
bart.eval()
bart.half()

In [44]:
#af = AF_Data_Generation_WP(bart_model_path)
for  file_type in ['train', 'valid']:
  lines_gt_stories = open(data_path2+'WP_{}.target'.format(file_type), 'r').readlines()
  manipulated_plots = open(data_path+'WP_{}_manipulated_plts'.format(file_type), 'r').readlines()
  lines_plots_prompts = open(data_path+'WP_{}_manipulpltsprompt'.format(file_type), 'r').readlines()
  plots_prmpts={}
  manipulated_plots_new,prompts = [],[]
  gt_stories,num_sents_gt_stories=[],[]
  for line in lines_plots_prompts:
  	plts = line.split('\n')[0].split('<EOL>')[1].strip()
  	if plts not in plots_prmpts:
  		plots_prmpts[plts] = line.split('<EOL>')[0].strip()
  for ind, plt in enumerate(manipulated_plots[:5]):
  	plt  = plt.split('\n')[0]
  	if plt in plots_prmpts.keys():
  		manipulated_plots_new.append(plt)
  		prompts.append(plots_prmpts[plt])#add two of manipulated plts' prompt (used in create_json_AF_input function)
  		prompts.append(plots_prmpts[plt])
  for ind_line, story in  enumerate(lines_gt_stories):
  	gt_story = lines_gt_stories[ind_line].split('\n')[0]
  	gt_stories.append(gt_story)
  	num_sents_gt_stories.append(len(gt_story.split('</s>')))
   
  for ind in range(num_negative_samples):  #Generate args.num_negative_samples different tsv files each including gt_stories as positive and generated negative stories as implausible ones
    #af.generate_implausible_stories(args, gt_stories, manipulated_plots_new, num_sents_gt_stories, ind,file_type) 	def generate_implausible_stories(self, args, gt_stories, manipulated_plots, num_sents_gt_stories, ind_file,file_type):
    file_ = open(os.path.join(data_path, '{}_pos_neg_stories_{}.tsv'.format(file_type, ind)), 'w')
    tsv_writer = csv.writer(file_, delimiter='\t', lineterminator='\n')
    count,indj = 1,0
    conv_lines,gt_stories_lines,num_sents_gt_lines = [],[],[]
    for ind_line, plots in  enumerate(manipulated_plots_new):
      plots = plots.strip()
      conv_lines.append(plots)
      gt_stories_lines.append(gt_stories[ind_line])
      num_sents_gt_lines.append(num_sents_gt_stories[ind_line])
      if count % batch_size == 0:
        with torch.no_grad():
          hyps = bart.sample(conv_lines, sampling =True, lenpen=2.0, max_len_b=max_len_b, min_len=min_len, sampling_topk=sampling_topk, temperature=temperature, beam=beam)	
          print("HYPS: ",hyps)
        for i, hypothesis in enumerate(hyps):
          hypothesis = hypothesis.strip().split('</s>')[:num_sents_gt_lines[i]]
          hypothesis = '</s>'.join(hypothesis).strip()
          if hypothesis[-1] not in ['.', '!', '?', '...', '?!']:
            hypothesis = hypothesis[:hypothesis.rfind('</s>')]
          tsv_writer.writerow([indj, '1', indj, gt_stories_lines[i]]);indj+=1
          tsv_writer.writerow([indj, '0', indj, hypothesis]);indj+=1	
        num_sents_gt_lines,gt_stories_lines,conv_lines=[],[],[]
      count +=1
    if conv_lines != []: #if finished before last batch reaches 20
      print("conv_lines: ",conv_lines)
      hyps = bart.sample(conv_lines, sampling=True, lenpen=2.0, max_len_b=max_len_b, min_len=min_len, sampling_topk=sampling_topk, temperature=temperature, beam=beam)
      print("HYPS: ",ind,"  ",hyps)
      for i, hypothesis in enumerate(hyps):
        hypothesis = hypothesis.strip().split('</s>')[:num_sents_gt_lines[i]]
        hypothesis = '</s>'.join(hypothesis)
        if hypothesis[-1] not in ['.', '!', '?', '...', '?!']:
          hypothesis = hypothesis[:hypothesis.rfind('</s>')]
        tsv_writer.writerow([indj, '1', indj, gt_stories_lines[i]]);indj+=1
        tsv_writer.writerow([indj, '0', indj, hypothesis]);indj+=1
    file_.close()

conv_lines:  ['standing\tabove\tdrop\twhole\t</s>\tblossoms\tsink\t</s>\t</s>\tcreatures\tscatter\trepulsiveness\t</s>\tgoes\t</s>\tpainted\telse\ts\tjust beautiful\t</s>\tloose\ta red cherry tree\t</s>\tup\ta\tthe front yard\t</s>\tIt\tstress\tclock#sit\tunder\ttake\tpart\t</s>\tblossoms\tfloat\t</s>\t</s>\tcreatures\tscatter\tbeauty\t</s>\tgoes\t</s>\tpainted\tit\ts\tjust beautiful\t</s>\tfind\ta red cherry tree\t</s>\tput\ta\tthe front yard\t</s>\tIt\trelax\twatch#standing\tabove\tdrop\twhole\t</s>\tblossoms\tsink\t</s>\t</s>\tcreatures\tscatter\trepulsiveness\t</s>\tgoes\t</s>\tpainted\telse\ts\tjust beautiful\t</s>\tloose\ta red cherry tree\t</s>\tup\ta\tthe front yard\t</s>\tIt\tstress\tclock', "stumbled\ttwirled\topen\tswallowing\tthe music and the voices\t</s>\tnascent\t</s>\trubbed\tforgot\tsubway was\t</s>\t</s>\t</s>\t</s>\tdrew\ta deep breath\tturned\tplowed\t</s>\t</s>\tunplanned\tanything\t</s>\t</s>\tsmacked\t</s>\tmaterialized\t</s>\t</s>\t</s>\t</s>\t</s>\tsaid\t</s>\t

/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size


HYPS:  0    ['*Suddenly , suddenly the ground shakes , and the watch# standing above will drop whole . </s> The flowers sink into the earth . </s> *The clock stops . </s> * The world goes quiet . </s> The world goes to sleep . </s> The flower is all painted white , everything else ’ s just beautiful . </s> I should loosen up a red cherry tree . </s> I should loosen up a large red umbrella in the front yard . </s> It ’ ll stress out the clock# sit under and take part in the moment . </s> The flowers float down to the ground . </s> *The world is quiet . </s> * The creatures scatter their beauty . </s> * The clock goes to sleep . </s> The flower is all painted white , nothing else ’ s just beautiful . </s> I should find a red cherry tree . </s> I should put it a in the front yard . </s> It ’ ll relax the clock # and watch the flowers . </s> The flowers sink into the earth . </s> *The clock goes to sleep . </s> * The world goes to sleep . </s> The flowers are all painted white , everything

In [42]:
af_input_dict["lines_valid_5"]

[]

In [ ]:
#af.create_json_AF_input(args, prompts),def create_json_AF_input(self, args, prompts):
af_input_dict = {}
for mode in ['train', 'valid']:
  for ind in range(num_negative_samples):
    af_input_dict['fr_{}_{}'.format(mode,ind)] = open(os.path.join(data_path, '{}_pos_neg_stories_{}.tsv'.format(mode, ind)), 'r')
    af_input_dict['lines_{}_{}'.format(mode,ind)] = af_input_dict['fr_{}_{}'.format(mode,ind)].readlines()

fw = open(os.path.join(data_path, json_file), 'w')
output = []
for mode in ['train', 'valid']:
  ind=0
  for i in range(0, len(af_input_dict['lines_{}_{}'.format(mode,ind)])-3, 2):
    output_text ={}
    gt_story = af_input_dict['lines_{}_{}'.format(mode,ind)][i].split('\t')[3].split('\n')[0].strip()
    gens=[]
    for ind in range(num_negative_samples):
      print(i+1,len(af_input_dict['lines_{}_{}'.format(mode,ind)]))
      if(len(af_input_dict['lines_{}_{}'.format(mode,ind)])==0):print(mode,ind,af_input_dict['lines_{}_{}'.format(mode,ind)])
      af_input_dict['gen_story{}'.format(ind)]=af_input_dict['lines_{}_{}'.format(mode,ind)][i+1].split('\t')[3].split('\n')[0].strip()
      gens.append(af_input_dict['gen_story{}'.format(ind)])
    output_text["ctx"] = prompts[i]
    output_text["gt_detok"] = gt_story
    output_text["gens"] = gens 
    output.append(output_text)
    ind=0		
fw.write('\n'.join(json.dumps(i, ensure_ascii=False) for i in output))

##Step 5 (convert to tsv)

In [48]:
globals()['{}'.format('train')]

KeyError: ignored

In [66]:
list_ctx[globals()[0:3]]

TypeError: ignored

In [79]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--data_path", type=str, default='Data/WP/WP_Eval/', help='data path for WP_Eval data')
#parser.add_argument("--output_AF", type=str, default='AF_output.json', help='The result file from AF')
#parser.add_argument("--train_output", type=str, default='AF_ManPlts_train.tsv', help='The training file resulted from AF applied on WP manipulated plots to be used for training the evaluator')
#parser.add_argument("--valid_output", type=str, default='AF_ManPlts_valid.tsv', help='The validation file resulted from AF applied on WP manipulated plots to be used for validating the evaluator')
#parser.add_argument("--test_output", type=str, default='AF_ManPlts_test.tsv', help='The testing file resulted from AF applied on WP manipulated plots to be used for testing the evaluator')
#args = parser.parse_args()
#data_path = '/content/drive/My Drive/Colab Notebooks/NLP_Project/base_project/Data_/WP/WP_Eval/'
data_path = '/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_ManPlts/'
data_path2 = '/content/drive/My Drive/Colab Notebooks/NLP_Project/Data_/WP/WP_AF_ManPlts/'
output_AF = "WP_AF_input.json"#'AF_output.json'

fr = open(os.path.join(data_path, output_AF), 'r')
fw_train = open(os.path.join(data_path2, 'AF_ManPlts_train.tsv'), 'w')
fw_valid = open(os.path.join(data_path2, 'AF_ManPlts_valid.tsv'), 'w')
fw_test = open(os.path.join(data_path2, 'AF_ManPlts_test.tsv'), 'w')
tsv_train = csv.writer(fw_train, delimiter='\t', lineterminator='\n')
tsv_valid = csv.writer(fw_valid, delimiter='\t', lineterminator='\n')
tsv_test = csv.writer(fw_test, delimiter='\t', lineterminator='\n')

list_ctx,list_gt,list_gens = [],[],[]
num_stories= 0
for line in fr:
  line = json.loads(line)
  list_ctx.append(line['ctx'])
  list_gt.append(line['gt_detok'])
  gens = []
  for l in line['gens']:
    gens.append(l)
  list_gens.append(gens)
	#sel_inds = line['assignment'][-1] #line['assignment'][-1] shows the index of most challenging generated stories based on the applied AF
	#for ind in sel_inds:
  #	gens.append(line['gens'][ind])

num_stories=len(list_ctx)
num_train_stories = math.ceil((60*num_stories)/100)
num_valid_stories = math.ceil((20*num_stories)/100)
num_test_stories = math.ceil((20*num_stories)/100)
tsv_out = {}
start_ind={'train':0, 'valid':num_train_stories, 'test': num_train_stories + num_valid_stories}
for mode in ['train', 'valid', 'test']:
	st_ind = start_ind['{}'.format(mode)]
	start_ind['{}_ctx'.format(mode)] = list_ctx[st_ind:st_ind+eval('num_{}_stories'.format(mode))]
	start_ind['{}_gt'.format(mode)] = list_gt[st_ind:st_ind+eval('num_{}_stories'.format(mode))]
	start_ind['{}_gens'.format(mode)] = list_gens[st_ind:st_ind+eval('num_{}_stories'.format(mode))]

for mode in ['train', 'valid', 'test']:
  line_ind = 0
  print(start_ind['{}_gt'.format(mode)])
  for ind, gt in enumerate(start_ind['{}_gt'.format(mode)]):
    print("cc",mode)
    eval('tsv_{}'.format(mode)).writerow([line_ind, 1, line_ind , gt])
    line_ind+=1
    for gen in start_ind['{}_gens'.format(mode)][ind]:
      eval('tsv_{}'.format(mode)).writerow([line_ind, 0, line_ind, gen])
      line_ind+=1

['I have two trees out in the yard that I always sit under and take part in a poorly recreated *hanami* . </s> The blossoms float and drift on the wind . </s> Ephemeral little creatures they are . </s> Beautiful ephemeral creatures that scatter their beauty all over the neighboring lawns and houses . </s> A good glass of alcohol always goes with it . </s> Red blossoms are painted all over my study and I find that it ’ s just as beautiful . </s> Maybe I should see if I can find a red cherry blossom tree . </s> I should put a footrest out in the front yard next spring too . </s> It ’ s so much more comfortable to relax with a footrest to watch the blossoms .', "Gabriel stumbled and twirled away from the door as it closed behind him , swallowing up the music and the voices . </s> The air was cool and crisp and purple in the dying streetlights . </s> He rubbed his eyes and tried to remember where the subway was . </s> Left or right ? Right ? Right , right ? He drew a deep breath , turned t